In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip freeze > requirements.txt

In [ ]:

# !pip install tensorflow scikit-learn

"""# Paths"""

rpsense_train_path = '/kaggle/input/rpsense-dataset/train'
rpsense_test_path = '/kaggle/input/rpsense-dataset/test'
rpsense_validation_path = '/kaggle/input/rpsense-dataset/validation'

classes = ['invalid', 'paper', 'rock', 'scissors']

"""# Import libraries"""

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, AveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the target image size and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32


In [ ]:

"""# Data Augmentation"""

# Data augmentation for training to improve model generalization
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,      
    rotation_range=20,              # Random rotation
    width_shift_range=0.2,          # Horizontal shift
    height_shift_range=0.2,         # Vertical shift
    shear_range=0.2,                # Shearing transformation
    zoom_range=0.2,                 # Zoom in/out
    horizontal_flip=True,           # Random horizontal flip
    fill_mode='nearest'             # Fill pixels after transformations
)

# For validation and testing: only normalize
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Show some augmented images
x_batch, y_batch = next(
    train_datagen.flow_from_directory(
        rpsense_train_path,
        target_size=(224, 224),
        batch_size=16
    )
)
# Plot 8 images from the batch
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(x_batch[i])
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:

# """Try connecting to TPU"""

# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("✅ Connected to TPU")
# except ValueError:
#     strategy = tf.distribute.get_strategy()  # Default strategy for CPU/GPU
#     print("⚠️ TPU not found, using", strategy)

"""
Try using GPU

"""
# Encountered NAN when using mirrored stratergy so shifting to 1 GPU only

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("✅ GPU detected:", physical_devices[0].name)
else:
    print("⚠️ No GPU found, using CPU")

strategy = tf.distribute.get_strategy()  # Automatically detects and uses GPU

print("✅ Using strategy:", strategy)




"""# Load dataset"""

# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    rpsense_train_path,             # Path to training data
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize all images
    batch_size=BATCH_SIZE,
    class_mode='categorical',       # For multi-class classification
    classes=classes,                 # Define class order manually
    # shuffle=True                    # Shuffle data for each epoch
)

# Load the validation dataset
validation_generator = validation_datagen.flow_from_directory(
    rpsense_validation_path,        # Path to validation data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    # shuffle=False
)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    rpsense_test_path,              # Path to test data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes
)

"""compile in stratergy"""

with strategy.scope():
    base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    )
    
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = Dense(len(classes), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(
        learning_rate=0.0001,
        clipnorm=1.0,  # Clip gradients by norm
    )
    
    model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )
model.summary()

"""# Configure Early Stopping and ModelCheckpoint

"""

early_stop = EarlyStopping(
    monitor='val_loss',       # Or 'val_accuracy'
    patience=5,               # Stop if no improvement after 5 epochs
    restore_best_weights=True  # Load best model weights at end
)

checkpoint = ModelCheckpoint(
    'best_mobilenetv2_rpsense_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose = 1
)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)


In [ ]:
len(model.layers)

In [ ]:
# a custom callback to monitor for NaN values during training
class NaNMonitor(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is not None:
            if np.isnan(logs.get('loss', 0)) or np.isnan(logs.get('accuracy', 0)):
                print(f"\n⚠️ NaN detected at batch {batch}!")
                print(f"Loss: {logs.get('loss')}, Accuracy: {logs.get('accuracy')}")
                self.model.stop_training = True

# Create NaN monitor callback
nan_monitor = NaNMonitor()

# Also add a custom callback to check model weights
class WeightMonitor(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Check if any weights are NaN
        for layer in self.model.layers:
            if hasattr(layer, 'weights') and layer.weights:
                for weight in layer.weights:
                    if tf.reduce_any(tf.math.is_nan(weight)):
                        print(f"\n⚠️ NaN weights detected in layer {layer.name} at epoch {epoch}")
                        self.model.stop_training = True
                        return

weight_monitor = WeightMonitor()

# Debug: Check model weights before training
print("Checking initial model weights...")
for layer in model.layers[-3:]:  # Check last 3 layers
    if hasattr(layer, 'weights') and layer.weights:
        for i, weight in enumerate(layer.weights):
            weight_values = weight.numpy()
            print(f"Layer {layer.name} weight {i}: shape={weight_values.shape}, "
                  f"min={weight_values.min():.6f}, max={weight_values.max():.6f}, "
                  f"mean={weight_values.mean():.6f}, has_nan={np.isnan(weight_values).any()}")

# Reset the generators to ensure clean state
train_generator.reset()
validation_generator.reset()

# Debug: Check for NaN values in data before training
print("Checking data for NaN values...")
x_batch, y_batch = next(train_generator)
print(f"Training batch shape: {x_batch.shape}")
print(f"Training batch min/max: {x_batch.min():.4f} / {x_batch.max():.4f}")
print(f"Training batch has NaN: {np.isnan(x_batch).any()}")
print(f"Training labels shape: {y_batch.shape}")
print(f"Training labels has NaN: {np.isnan(y_batch).any()}")

x_val_batch, y_val_batch = next(validation_generator)
print(f"Validation batch shape: {x_val_batch.shape}")
print(f"Validation batch min/max: {x_val_batch.min():.4f} / {x_val_batch.max():.4f}")
print(f"Validation batch has NaN: {np.isnan(x_val_batch).any()}")
print(f"Validation labels has NaN: {np.isnan(y_val_batch).any()}")

# Reset generators again after debugging
train_generator.reset()
validation_generator.reset()

# Try a test forward pass to check for NaN
print("Testing forward pass...")
test_prediction = model.predict(x_batch[:1], verbose=0)
print(f"Test prediction shape: {test_prediction.shape}")
print(f"Test prediction has NaN: {np.isnan(test_prediction).any()}")
print(f"Test prediction values: {test_prediction[0]}")


In [ ]:
# Train the model using the training and validation data

EPOCHS = 50
# Train using only 1000 images for each class.
# train_steps = 1000 // BATCH_SIZE
# val_steps = 300 // BATCH_SIZE

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint, reduce_lr, nan_monitor, weight_monitor],
    verbose=1
    # validation_steps=val_steps,
    # steps_per_epoch=train_steps
)

model_save_path = '/kaggle/working/mobilenetv2_rpsense.h5'
# Save the trained model to disk
model.save(model_save_path)

In [ ]:

"""# Evaluate on Test set"""

# Evaluate the trained model on the test dataset
test_loss, test_acc = model.evaluate(test_eval_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
train_eval_generator = train_datagen.flow_from_directory(
    rpsense_train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False  
)

# Load the test dataset
test_eval_generator = test_datagen.flow_from_directory(
    rpsense_test_path,              # Path to test data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False  
)

# Get true labels
y_true = test_eval_generator.classes

# Get predicted labels
y_pred_probs = model.predict(test_eval_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

# Class names from the generator
class_names = list(test_eval_generator.class_indices.keys())

# Confusion Matrix for Test dataset
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Test Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Confusion Matrix for Train dataset
y_train_true = train_eval_generator.classes
y_train_pred = np.argmax(model.predict(train_eval_generator), axis=1)

cm_train = confusion_matrix(y_train_true, y_train_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Train Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print("Classification Report (Test):")
print(classification_report(y_true, y_pred, target_names=class_names))

print("Classification Report (Train):")
print(classification_report(y_train_true, y_train_pred, target_names=class_names))

In [ ]:
# Evaluate the trained model on the train dataset
train_loss, train_acc = model.evaluate(train_eval_generator)
print(f"✅ Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")


In [ ]:
print(train_generator.class_indices)
print(test_generator.class_indices)
# Class names from the generator
class_names = list(test_generator.class_indices.keys())
class_names

# there was mismatch between true and predicted labels, bcoz shuffle = True by default.

In [ ]:
import pickle
with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/working/mobilenetv2_rpsense.h5')

for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

# Evaluate the trained model on the test dataset
test_loss, test_acc = model.evaluate(test_eval_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


In [ ]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Smaller LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



history = model.fit(
    train_generator,  # or new train_eval_generator if you want shuffle=False
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]  # if you want
)


# Fine Tuning

In [ ]:
# Fine-tuning script for RPS Classification Model
# This script loads the pre-trained model and performs fine-tuning of layers after 100

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Paths and constants
rpsense_train_path = '/kaggle/input/rpsense-dataset/train'
rpsense_test_path = '/kaggle/input/rpsense-dataset/test'
rpsense_validation_path = '/kaggle/input/rpsense-dataset/validation'
model_path = '/kaggle/input/rpsensemobilenetv2/keras/default/1/mobilenetv2_rpsense.h5'

classes = ['invalid', 'paper', 'rock', 'scissors']
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Set up strategy for GPU/TPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("✅ GPU detected:", physical_devices[0].name)
else:
    print("⚠️ No GPU found, using CPU")

strategy = tf.distribute.get_strategy()
print("✅ Using strategy:", strategy)

# Data augmentation for training to improve model generalization
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,      
    rotation_range=15,              # Reduced from 20 to prevent overfitting
    width_shift_range=0.1,          # Reduced from 0.2
    height_shift_range=0.1,         # Reduced from 0.2
    shear_range=0.1,                # Reduced from 0.2
    zoom_range=0.1,                 # Reduced from 0.2
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],    # Add brightness variation
    fill_mode='nearest'
)

# For validation and testing: only normalize
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    rpsense_train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    rpsense_validation_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    rpsense_test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False
)

with strategy.scope():
    # Load the pre-trained model
    print("Loading pre-trained model...")
    model = load_model(model_path)
    print("✅ Model loaded successfully")
    
    # Print model summary to understand the architecture
    print(f"Total layers in model: {len(model.layers)}")
    for i, layer in enumerate(model.layers):
        print(f"Layer {i}: {layer.name} - Trainable: {layer.trainable}")
    
    # Freeze early layers, unfreeze later layers for fine-tuning
    # You can adjust these numbers based on your model architecture
    freeze_until = 100  # Freeze first 100 layers
    
    for i, layer in enumerate(model.layers):
        if i < freeze_until:
            layer.trainable = False
        else:
            layer.trainable = True
    
    print(f"✅ Layers 0-{freeze_until-1} frozen, layers {freeze_until}+ unfrozen")
    
    # Count trainable parameters
    trainable_params = sum([np.prod(layer.trainable_weights[0].shape) for layer in model.layers if layer.trainable_weights])
    print(f"Trainable parameters: {trainable_params:,}")

    optimizer = Adam(
        learning_rate=1e-5,
        clipnorm=1.0,  # Clip gradients by norm
    )
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

# Setup callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_finetuned_mobilenetv2_rpsense_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# Training parameters
FINETUNE_EPOCHS = 20

print(f"\n🔥 Starting fine-tuning for {FINETUNE_EPOCHS} epochs...")
history = model.fit(
    train_generator,
    epochs=FINETUNE_EPOCHS,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint, reduce_lr],
    verbose=1
)

# Save the fine-tuned model
finetuned_model_path = '/kaggle/working/finetuned_after100layers_mobilenetv2_rpsense.h5'
model.save(finetuned_model_path)
print(f"✅ Fine-tuned model saved to: {finetuned_model_path}")

In [ ]:
import pickle
with open('finetune_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


# Fine tune evaluation

In [ ]:
print("\n📊 Evaluating fine-tuned model on test set...")
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"✅ Fine-tuned Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()



# Generate predictions and confusion matrix
print("\n🔍 Generating predictions and confusion matrix...")
test_generator.reset()
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix - Fine-tuned Model')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Print classification report
print("\n📋 Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=classes))

# Compare with original model performance
print("\n📈 Performance Comparison:")
print("Original Model - Test Accuracy: ~89%")
print(f"Fine-tuned Model - Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

improvement = (test_acc - 0.89) * 100
if improvement > 0:
    print(f"✅ Improvement: +{improvement:.2f}%")
else:
    print(f"⚠️ Change: {improvement:.2f}%")

print("\n🎯 Fine-tuning completed!")

In [ ]:
train_eval_generator = train_datagen.flow_from_directory(
    rpsense_train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False  
)

class_names = list(train_eval_generator.class_indices.keys())

# Confusion Matrix for Train dataset
y_train_true = train_eval_generator.classes
y_train_pred = np.argmax(model.predict(train_eval_generator), axis=1)

cm_train = confusion_matrix(y_train_true, y_train_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Train Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

print("Classification Report (Train):")
print(classification_report(y_train_true, y_train_pred, target_names=class_names))

In [ ]:
# Evaluate the trained model on the train dataset
train_loss, train_acc = model.evaluate(train_eval_generator)
print(f"✅ Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")


In [ ]:
# Evaluate the trained model on the train dataset
valid_loss, valid_acc = model.evaluate(validation_generator)
print(f"✅ Valid Accuracy: {valid_acc:.4f}, Valid Loss: {valid_loss:.4f}")


In [ ]:
# Export model in TensorFlow SavedModel format (for TF Serving, TFLite, etc.)
saved_model_dir = '/kaggle/working/finetuned_after100layers_mobilenetv2_rpsense_savedmodel'
model.export(saved_model_dir)

print(f"✅ Exported model saved to: {saved_model_dir}")

import shutil

# Define zip path
zip_path = '/kaggle/working/finetuned_after100layers_mobilenetv2_rpsense_savedmodel.zip'

# Zip the exported model folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', saved_model_dir)

print(f"✅ Zipped exported model: {zip_path}")